## Brief Projet : 

Vous êtes un data analyst ,vous travaillez en bénévolat pour la médiathèque de la ville qui a mis en place un site web pour la vente de certains livres de son stock.. et elle souhaite analyser les caractéristiques sa clientèle pour mieux comprendre les tendances de leurs reservations/achats de livres. 

La médiathèque vous donne accès à leur site : http://books.toscrape.com/index.html. et souhaite que vous collectez/analysez leurs données.

**Objectifs**  :
- Découvrir les bases du scraping web avec requests et BeautifulSoup.

- Structurer les données avec pandas.

- Utiliser SQLite pour stocker localement.

- Créer une API FastAPI avec 3 routes simples.


**Étapes du projet** :

1. **Collecte de données** : 
* Utiliser la bibliothèque `requests` pour envoyer des requêtes HTTP au site web qui répertorie les livres. Vous récupérez le contenu HTML de la page web.

* A l'aide de la bibliothèque `Beautiful Soup`, analyser le contenu HTML du site et extraire les informations pertinentes: parcourir le code HTML, identifier les balises cibles (qui contiennent les données sur les livres, telles que `<div>` ou `<li>` ) et extraire les informations pertinentes telles que le nom du livre, la catégorie, la note moyenne des avis, le nombre de livres en stock, le prix etc.

2. **Nettoyage et préparation des données** : nettoyer les valeurs, convertir les types de données si nécessaire, gérer les valeurs manquantes, etc.

3. **Stockage des données** : 
* Proposer une modélisation de base de données SQL adaptée. 
* Créer le schéma de la base de données , les différentes tables pour stocker les données propres sur les livres.

4. **Analyse des données** : 

* Faire une analyse exploratoire des données : identification de KPIs pertinents ,création de graphiques, le calcul de statistiques descriptives, l'identification de tendances, etc => pour aider la médiathèque à mieux faire son étude de clientèle. 



### Importer les librairies

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Inspecter (click droit => Inspecter) sur le site web que l'on doit "webscrapper" est la porte d'entrée, 
le commencement de tout projet de web scrapping

### Collecte de données

####  Récupérer les livres de la première page

In [2]:
# URL de la page principale
url = "http://books.toscrape.com/"



In [3]:
# URL de la page principale
url = "http://books.toscrape.com/"
#url="http://books.toscrape.com/catalogue/page-1.html"
# Envoyer une requête GET à l'URL
response = requests.get(url)

# Vérifier si la requête s'est bien déroulée
if response.status_code == 200:
    # Analyser le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver tous les liens de livres sur la page principale
    book_links = soup.find_all('h3')

    
    # Parcourir les liens de livres
    for link in book_links:
        # Extraire le titre du livre
        title = link.a.attrs['title']

        # Accéder à la page de détails du livre
        book_url = url + link.a.attrs['href']
        book_response = requests.get(book_url)
        book_soup = BeautifulSoup(book_response.text, 'html.parser')

        # Extraire le prix du livre
        price = book_soup.find('p', class_='price_color').text

        # Extraire la notation du livre
        rating = book_soup.find('p', class_='star-rating')['class'][1]

        # Imprimer les informations extraites
        print(f"Titre : {title}")
        print(f"Price : {price}")
        print(f"Rating : {rating}")


Titre : A Light in the Attic
Price : Â£51.77
Rating : Three
Titre : Tipping the Velvet
Price : Â£53.74
Rating : One
Titre : Soumission
Price : Â£50.10
Rating : One
Titre : Sharp Objects
Price : Â£47.82
Rating : Four
Titre : Sapiens: A Brief History of Humankind
Price : Â£54.23
Rating : Five
Titre : The Requiem Red
Price : Â£22.65
Rating : One
Titre : The Dirty Little Secrets of Getting Your Dream Job
Price : Â£33.34
Rating : Four
Titre : The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Price : Â£17.93
Rating : Three
Titre : The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Price : Â£22.60
Rating : Four
Titre : The Black Maria
Price : Â£52.15
Rating : One
Titre : Starving Hearts (Triangular Trade Trilogy, #1)
Price : Â£13.99
Rating : Two
Titre : Shakespeare's Sonnets
Price : Â£20.66
Rating : Four
Titre : Set Me Free
Price : Â£17.46
Rating : Five
Titre : Scott Pilgrim's Precious Little Life (Scott Pilgrim

In [4]:
print(link)

<h3><a href="catalogue/its-only-the-himalayas_981/index.html" title="It's Only the Himalayas">It's Only the Himalayas</a></h3>


In [5]:
print(link.find('a').get('title'))

It's Only the Himalayas


In [6]:
print(link.a.attrs['title'])

It's Only the Himalayas


In [7]:
print(link.find('a').get_text())

It's Only the Himalayas


In [8]:
print(book_links)

[<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>, <h3><a href="catalogue/tipping-the-velvet_999/index.html" title="Tipping the Velvet">Tipping the Velvet</a></h3>, <h3><a href="catalogue/soumission_998/index.html" title="Soumission">Soumission</a></h3>, <h3><a href="catalogue/sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>, <h3><a href="catalogue/sapiens-a-brief-history-of-humankind_996/index.html" title="Sapiens: A Brief History of Humankind">Sapiens: A Brief History ...</a></h3>, <h3><a href="catalogue/the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a></h3>, <h3><a href="catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html" title="The Dirty Little Secrets of Getting Your Dream Job">The Dirty Little Secrets ...</a></h3>, <h3><a href="catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html

#### Récupérer la liste des catégories

In [9]:
# Créer une liste avec les liens href de toutes les catégories
categories_urls=[]
#To Be Create
for link in soup.find_all('a', href=True):
    if 'catalogue/category/books' in link['href']:
        categories_urls.append(link['href'])
    
categories_urls

['catalogue/category/books_1/index.html',
 'catalogue/category/books/travel_2/index.html',
 'catalogue/category/books/mystery_3/index.html',
 'catalogue/category/books/historical-fiction_4/index.html',
 'catalogue/category/books/sequential-art_5/index.html',
 'catalogue/category/books/classics_6/index.html',
 'catalogue/category/books/philosophy_7/index.html',
 'catalogue/category/books/romance_8/index.html',
 'catalogue/category/books/womens-fiction_9/index.html',
 'catalogue/category/books/fiction_10/index.html',
 'catalogue/category/books/childrens_11/index.html',
 'catalogue/category/books/religion_12/index.html',
 'catalogue/category/books/nonfiction_13/index.html',
 'catalogue/category/books/music_14/index.html',
 'catalogue/category/books/default_15/index.html',
 'catalogue/category/books/science-fiction_16/index.html',
 'catalogue/category/books/sports-and-games_17/index.html',
 'catalogue/category/books/add-a-comment_18/index.html',
 'catalogue/category/books/fantasy_19/index.

#### Récupérer les livres de toutes les pages  

In [10]:

# Fonction pour extraire les informations d'une page donnée
def extract_book_info(page_url):
    books=[]
    response = requests.get(page_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        #livres = soup.find_all(class_='product_pod')
        livres = soup.find_all('article', {'class': 'product_pod'})
        #print(livres)
        for livre in livres:
            # Extraire le titre du livre
            titre = livre.h3.a.attrs['title']
            
            rating = livre.p['class'][1]

            prix = livre.find('p', class_='price_color').text

            disponibilite = livre.find('p', class_='instock availability').text.strip()

            books.append({'title': titre, 'rating': rating, 'price': prix, 'availability': disponibilite})
        df_books=pd.DataFrame(books)
            
            
    return df_books
             
extract_book_info(url)



,title,rating,price,availability
0,A Light in the Attic,Three,Â£51.77,In stock
1,Tipping the Velvet,One,Â£53.74,In stock
2,Soumission,One,Â£50.10,In stock
3,Sharp Objects,Four,Â£47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,Â£54.23,In stock
5,The Requiem Red,One,Â£22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,Four,Â£33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,Three,Â£17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,Four,Â£22.60,In stock
9,The Black Maria,One,Â£52.15,In stock


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_book_info(base_url, num_pages=20):
    books = []
    
    for page in range(1, num_pages + 1):
        page_url = f"{base_url}/catalogue/page-{page}.html"
        try:
            response = requests.get(page_url, timeout=10)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                livres = soup.find_all('article', class_='product_pod')
                
                if not livres:
                    print(f"Plus de livres trouvés à la page {page}. Arrêt.")
                    break
                
                for livre in livres:
                    try:
                        titre = livre.h3.a.attrs['title']
                        rating = livre.p['class'][1] if livre.p['class'] else 'N/A'
                        prix = livre.find('p', class_='price_color').text if livre.find('p', class_='price_color') else 'N/A'
                        disponibilite = livre.find('p', class_='instock availability').text.strip() if livre.find('p', class_='instock availability') else 'N/A'
                        
                        books.append({
                            'title': titre,
                            'rating': rating,
                            'price': prix,
                            'availability': disponibilite,
                            'page': page
                        })
                    except (AttributeError, IndexError) as e:
                        print(f"Erreur lors de l'extraction d'un livre à la page {page}: {e}")
                        continue
                
                print(f"Page {page} traitée avec succès")
            else:
                print(f"Erreur HTTP {response.status_code} pour la page {page}")
                continue
                
        except requests.RequestException as e:
            print(f"Erreur de connexion pour la page {page}: {e}")
            continue
    
    df_books = pd.DataFrame(books)
    return df_books

# Exemple d'utilisation
base_url = "http://books.toscrape.com"  # Remplacez par l'URL réelle si différente
df = extract_book_info(base_url, num_pages=20)

if not df.empty:
    df.to_csv('books_data.csv', index=False)
    print(f"Données extraites pour {len(df)} livres et sauvegardées dans 'books_data.csv'")
else:
    print("Aucune donnée extraite")



Page 1 traitée avec succès
Page 2 traitée avec succès
Page 3 traitée avec succès
Page 4 traitée avec succès
Page 5 traitée avec succès
Page 6 traitée avec succès
Page 7 traitée avec succès
Page 8 traitée avec succès
Page 9 traitée avec succès
Page 10 traitée avec succès
Page 11 traitée avec succès
Page 12 traitée avec succès
Page 13 traitée avec succès
Page 14 traitée avec succès
Page 15 traitée avec succès
Page 16 traitée avec succès
Page 17 traitée avec succès
Page 18 traitée avec succès
Page 19 traitée avec succès
Page 20 traitée avec succès
Données extraites pour 400 livres et sauvegardées dans 'books_data.csv'


In [12]:
# Lecture du fichier CSV
df = pd.read_csv('books_data.csv')

# Affichage des 20 premiers résultats
print("20 premiers résultats :")
print(df.head(20))

# Affichage des 20 derniers résultats
print("\n20 derniers résultats :")
print(df.tail(20))

20 premiers résultats :
                                                title rating    price  \
0                                A Light in the Attic  Three  Â£51.77   
1                                  Tipping the Velvet    One  Â£53.74   
2                                          Soumission    One  Â£50.10   
3                                       Sharp Objects   Four  Â£47.82   
4               Sapiens: A Brief History of Humankind   Five  Â£54.23   
5                                     The Requiem Red    One  Â£22.65   
6   The Dirty Little Secrets of Getting Your Dream...   Four  Â£33.34   
7   The Coming Woman: A Novel Based on the Life of...  Three  Â£17.93   
8   The Boys in the Boat: Nine Americans and Their...   Four  Â£22.60   
9                                     The Black Maria    One  Â£52.15   
10     Starving Hearts (Triangular Trade Trilogy, #1)    Two  Â£13.99   
11                              Shakespeare's Sonnets   Four  Â£20.66   
12                         

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_book_info(page_url):
    books = []
    response = requests.get(page_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        livres = soup.find_all('article', {'class': 'product_pod'})
        
        for livre in livres:
            # Extraire le titre (attribut title de la balise <a> dans <h3>)
            titre = livre.find('h3').find('a')['title'] if livre.find('h3').find('a') else 'N/A'

            # Extraire le rating (classe de la balise <p class="star-rating ...">)
            rating = livre.find('p', class_='star-rating')['class'][1] if livre.find('p', class_='star-rating') else 'N/A'

            # Extraire le prix (texte de la balise <p class="price_color">)
            prix = livre.find('p', class_='price_color').text.strip() if livre.find('p', class_='price_color') else 'N/A'

            # Extraire la disponibilité (texte de la balise <p class="instock availability">)
            disponibilite = livre.find('p', class_='instock availability').text.strip() if livre.find('p', class_='instock availability') else 'N/A'

            books.append({'title': titre, 'rating': rating, 'price': prix, 'availability': disponibilite})
        
        df_books = pd.DataFrame(books)
    else:
        # Si la requête échoue, retourner un DataFrame vide
        df_books = pd.DataFrame(columns=['title', 'rating', 'price', 'availability'])
    
    return df_books
    

print(extract_book_info("http://books.toscrape.com/catalogue/page-1.html"))  

                                                title rating    price  \
0                                A Light in the Attic  Three  Â£51.77   
1                                  Tipping the Velvet    One  Â£53.74   
2                                          Soumission    One  Â£50.10   
3                                       Sharp Objects   Four  Â£47.82   
4               Sapiens: A Brief History of Humankind   Five  Â£54.23   
5                                     The Requiem Red    One  Â£22.65   
6   The Dirty Little Secrets of Getting Your Dream...   Four  Â£33.34   
7   The Coming Woman: A Novel Based on the Life of...  Three  Â£17.93   
8   The Boys in the Boat: Nine Americans and Their...   Four  Â£22.60   
9                                     The Black Maria    One  Â£52.15   
10     Starving Hearts (Triangular Trade Trilogy, #1)    Two  Â£13.99   
11                              Shakespeare's Sonnets   Four  Â£20.66   
12                                        Set Me Fr

In [14]:
response = requests.get('http://books.toscrape.com/catalogue/page-2.html')
if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        #livres = soup.find_all(class_='product_pod')
        livres = soup.find_all('article', {'class': 'product_pod'})
        print(livres)

[<article class="product_pod">
<div class="image_container">
<a href="in-her-wake_980/index.html"><img alt="In Her Wake" class="thumbnail" src="../media/cache/5d/72/5d72709c6a7a9584a4d1cf07648bfce1.jpg"/></a>
</div>
<p class="star-rating One">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="in-her-wake_980/index.html" title="In Her Wake">In Her Wake</a></h3>
<div class="product_price">
<p class="price_color">Â£12.84</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>, <article class="product_pod">
<div class="image_container">
<a href="how-music-works_979/index.html"><img alt="How Music Works" class="thumbnail" src="../media/cache/5c/c8/5cc8e107246cb478960d4f0aba1e1c8e.jpg"/></a>
</div>
<p class="star-rating

In [15]:
p=extract_book_info('http://books.toscrape.com/catalogue/page-2.html')

In [16]:
p.head()

,title,rating,price,availability
0,In Her Wake,One,Â£12.84,In stock
1,How Music Works,Two,Â£37.32,In stock
2,Foolproof Preserving: A Guide to Small Batch J...,Three,Â£30.52,In stock
3,Chase Me (Paris Nights #2),Five,Â£25.27,In stock
4,Black Dust,Five,Â£34.53,In stock


In [18]:
p.shape

(20, 4)

In [ ]:
# Une Boocle pour parcourir ttes les pages
#TBD



In [19]:

df_books_.shape

NameError: name 'df_books_' is not defined

In [ ]:
df_books_.head()

,title,rating,price,availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.10,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock


In [ ]:
print('All_books:',type(all_books),'de ',len(all_books),'pages,  avec' ,all_books[0].shape[0], 'livres par page')

All_books: <class 'list'> de  50 pages,  avec 20 livres par page


#### Récupérer les livres de toutes les pages avec leurs  catégories

In [ ]:
#TBD

In [ ]:
print(len(categories), 'categories \n', 'répartie comme suit:', df_categories)

51 categories 
 répartie comme suit:               category  nb_books
0                Books      1020
1               Travel        11
2              Mystery        52
3   Historical Fiction        46
4       Sequential Art        95
5             Classics        19
6           Philosophy        11
7              Romance        55
8       Womens Fiction        17
9              Fiction        85
10           Childrens        49
11            Religion         7
12          Nonfiction       130
13               Music        13
14             Default       172
15     Science Fiction        16
16    Sports and Games         5
17       Add a comment        87
18             Fantasy        68
19           New Adult         6
20         Young Adult        74
21             Science        14
22              Poetry        19
23          Paranormal         1
24                 Art         8
25          Psychology         7
26       Autobiography         9
27           Parenting         1
28    

In [ ]:
#Save data into csv file
book_infos.to_csv('books.csv',index=False,header=True)

### Nettoyage et préparation des données

In [ ]:
df1=pd.read_csv('books.csv')
df1.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books


#### Comprendre la structure des données + Analyse descriptive 


* Etudier la structure du DataFrame : head(), info(),describe() , observer les diff types de données, les valeurs manquantes,les doublons, les statistiques sommaires, etc.


In [ ]:
#TBD

##### Gestion des valeurs manquantes

* Rechercher des valeurs manquantes  à l'aide de la méthode `.isnull()` ou `.isna()`. 

=> Si vous en trouvez, vous pouvez décider de les supprimer ou de les remplacer par des valeurs appropriées.

In [ ]:
 #Vérifier les valeurs manquantes
#TBD

#####  Gestion des doublons

* Utiliser la méthode `drop_duplicates()` pour supprimer les lignes dupliquées.


In [ ]:
# Vérifier les doublons dans le DataFrame
#TBD

#### Conversion de types de données

* Vérifier que les colonnes "rating" et "price", availability ont les types de données appropriés. 



In [ ]:
#TBD

### Stockage de données 

* Proposer une modélisation cohérente et créer la BDD dans votre SGBDR

### Analyse exploratoire et Visualisation des données

* faire une analyse exploratoire des données pour mieux comprendre la distribution des valeurs dans chaque colonne, identifier des tendances ou des valeurs aberrantes potentielles..

#### Analyse univariée, Visualiser les données

* Distribution de la variable 'price' =>  Création d'un histogramme pour le prix

In [ ]:
#TBD


* Distribution de la variable 'availability' 

In [ ]:
#TBD

* Distribution de la variable 'rating'

In [ ]:
#TBD

#### Analyser les categories de livres 

* comparer la catégorie 'Books' avec les reste des categories ..


In [ ]:
#TBD



* Création d'un diagramme à barres pour la répartition des livres par catégorie


In [ ]:
#TBD

#### Identifier les valeurs aberrantes (outliers)

* Création d'un boxplot pour le prix par catégorie


In [ ]:
#TBD

## Bonus:

###  Proposer d'autres axes d'analyse

### continuer le scraping du site en récupérant plus d'informations sur les livres..